In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import copy

In [5]:
class TicTacToe:
    def __init__(self):
        self.board = [[" " for _ in range(4)] for _ in range(4)]
        self.current_player = -1


    def check_draw(self):
        for row in self.board:
            if " " in row:
                return False
        return True

    def print_board(self):
        # Prints a GUI-like representation of the board
        print("┌───┬───┬───┬───┐")
        for i, row in enumerate(self.board):
            print("│ " + " │ ".join(row) + " │")
            if i < 3:
                print("├───┼───┼───┼───┤")
        print("└───┴───┴───┴───┘")

    def check_winner(self, player):
        for row in self.board:
            if all([cell == player for cell in row]):
                return True
        for col in range(4):
            if all([self.board[row][col] == player for row in range(4)]):
                return True
        if all([self.board[i][i] == player for i in range(4)]) or all(
            [self.board[i][3 - i] == player for i in range(4)]
        ):
            return True
        return False

    def step(self, state):
        '''
        Returns (new_state, current_player, done, reward)
        '''
        row = int(state / 4)
        col = int(state % 4)

        current_player_symbol = " "
        if self.current_player == -1:
            current_player_symbol = "X"
        else:
            current_player_symbol = "O"

        if self.board[row][col] == " ":
            self.board[row][col] = current_player_symbol

        if self.check_winner("O"):
            return self.board, self.current_player, True, -1
        elif self.check_winner("X"):
            return self.board, self.current_player, True, 1
        elif self.check_draw():
            return self.board, self.current_player, True, 0

        self.current_player *= -1

        return self.board, self.current_player, False, 0

In [3]:
# Initialized as a random policy for player 1

def policy_player1(board):

    possible_actions = []

    for i in range(4):
        for j in range(4):
            if board[i][j] == " ":
                possible_actions.append(i*4 + j)


    return random.choice(possible_actions)

# Initialized as a random policy for player 2
def policy_player2(board):

    possible_actions = []

    for i in range(4):
        for j in range(4):
            if board[i][j] == " ":
                possible_actions.append(i*4 + j)

    return random.choice(possible_actions)

In [4]:
from enum import Enum

class State(Enum):
    X = -1
    O = 1
    DRAW = 0
    NOT_TERMINAL = 2

def has_player_won(board, player):

    # List down all win conditions for a 3x3 board
    win_conditions = [
            (0, 1, 2, 3), (4, 5, 6, 7), (8, 9, 10, 11), (12, 13, 14, 15),   # horizontal
            (0, 4, 8, 12), (1, 5, 9, 13), (2, 6, 10, 14), (3, 7, 11, 15),    # vertical
            (0, 5, 10, 15), (3, 6, 9, 12)                                   # diagonal
    ]

    for condition in win_conditions:
        if all(board[i] == player for i in condition):
            return True
        
    return False

def is_board_terminal(board):
    '''
    Checks if the board is in a terminal state
    '''
    # Check if X or O have won
    if has_player_won(board, State.X.value):
        return State.X.value
    elif has_player_won(board, State.O.value):
        return State.O.value
    
    # If the board is full, but there's no win
    if 0 not in board:
        return State.DRAW.value
    
    # If the board is not full
    return State.NOT_TERMINAL.value

def reward_function(board):
    '''
    Returns the reward for the current board
    '''

    state_value = is_board_terminal(board)

    if state_value != State.NOT_TERMINAL.value:
        return state_value # could be a winning condition or a terminal draw
    else:
        return 0
    
def check_invalid_after_win(board):
    '''
    A utility function that checks if the board is in an invalid state after a win
    This can happen because X won and then O played again
    There are two cases to check if a winning satate is valid:
    1. If X has won, then there should be one more X than O
    2. If O has won, then there should be the same number of X and O
    3. If the game is a draw, then there should be one more X than O
    '''
    # Check if there is a winner
    has_x_won = has_player_won(board, State.X.value)
    has_o_won = has_player_won(board, State.O.value)
    state_value = is_board_terminal(board)

    # It is not possible that both have won
    if has_x_won and has_o_won:
        return True

    # If only X has won
    elif has_x_won:
        # Check if there is one more X than O
        if sum(board) == -1:
            return False
        else:
            return True
        
    # If only O has won
    elif has_o_won:
        # Check if there is the same number of X and O
        if sum(board) == 0:
            return False
        else:
            return True
    
    # If there is a draw then there should be one more X than O
    elif state_value == State.DRAW.value:
        # Should be one more X than O
        if sum(board) == -1:
            return False
        else:
            return True

    else:
        # If there is no winner, then the board is valid
        return False

def generate_boards(board, index=0):
    all_boards = []

    # If the board is complete, add it to the list
    if index == len(board):

        # Check if the sum of the board is -1 or 0
        valid_sum = (sum(board) in [-1, 0])
        valid_win = (not check_invalid_after_win(board))
        if valid_sum and valid_win:
            all_boards.append(board.copy())
        return all_boards

    # Try placing X, O, or leaving the spot empty
    for value in [-1, 0, 1]:
        board[index] = value
        all_boards.extend(generate_boards(board, index + 1))

    return all_boards

# states = generate_boards([0 for _ in range(16)])
# states = list(set([tuple(board) for board in states]))
# states = [list(board) for board in states]

states = np.load("boards.npy")
states.shape

(9716664, 16)

In [ ]:
def get_actions(state):
    '''
    Return possible actions
    '''
    return [i for i, p in enumerate(state) if p == 0]    

def get_reward(state):
    '''
    Return reward for state
    '''
    reward = is_board_terminal(state)
    reward = 0 if reward == State.NOT_TERMINAL.value else reward

    return reward

def get_turn(state):
    '''
    Return whose turn it is
    '''
    # -1 is X and 1 is O
    # If the sum is -1, then it is O's turn
    return 1 if sum(state) else -1

def Qlearning(states, num_episodes=10000, gamma=0.99, alpha=0.5, epsilon=1):
    '''
    Performs Value Iteration and returns the final Value table, Q-table and policies
    '''

    # Initialise tables
    q_table = {tuple(state): {action: 0 for action in get_actions(state) if is_board_terminal(state) == State.NOT_TERMINAL.value} for state in states}

    for _ in range(num_episodes):
        done = False        

        # Randomly select a state
        state = random.choice(states)
        to_play = get_turn(state)

        while not done:

            # Sample an action using epsilon-greedy
            if np.random().random() < epsilon:
                action = random.choice(get_actions(state))
            else:
                action = max(q_table[tuple(state)], key=q_table[tuple(state)].get)

            # Take the action
            next_state = state.copy()
            next_state[action] = to_play

            




# v_table, q_table, policy = value_iteration(states, turns, max_iterations=100, gamma=0.99)

In [186]:
def play_one_game(policy_player1, policy_player2):
    tictactoe = TicTacToe()

    terminated = 0
    board = [[" " for _ in range(4)] for _ in range(4)]

    for i in range(8):
        for turn in [-1, 1]:
            action = 0
            if turn == -1:
                action = policy_player1(board)
            else:
                action = policy_player2(board)

            board, player, terminated, reward = tictactoe.step(action)

            # Uncomment this if you want to see the board
            tictactoe.print_board()

            if terminated:
                break

    return -1*reward # This is the player who won


In [149]:
def run_alternating_games(games=10):
    results = []
    for i in range(games):
        for j in range(2):
            if j==0:
                winner = play_one_game(policy_player1, policy_player2)

                match winner:
                    case -1:
                        results.append(1)
                    case 1:
                        results.append(2)
                    case 0:
                        results.append(0)

            if j==1:
                winner = play_one_game(policy_player2, policy_player1)

                match winner:
                    case -1:
                        results.append(2)
                    case 1:
                        results.append(1)
                    case 0:
                        results.append(0)


    return results

In [183]:
results = run_alternating_games(1000)
print("Draws: ", results.count(0))
print("Player 1 Wins:", results.count(1))
print("Player 2 Wins:", results.count(2))

Draws:  845
Player 1 Wins: 577
Player 2 Wins: 578


I have created two functions that randomly select any action from the available actions from the board. Your team will have to create such a function that outputs the optimal action given a particular board state. This a similar kind of code I will be using on competition day when your function will play against an opponent's functions for perhaps a 1000 games. 

I will pass your and your opponent's function into the run alternating games function for maybe 1000 games to see who won more games. That person will be the winner of the match. I think it's a reliable method to compare policies. Run them by each for 1000s of games and see what policy wins the most games.

You have to solve this part using **Q Learning**